In [6]:
from datetime import datetime
import requests, json, os
from typing import Dict, Annotated, Optional, List, Union
from dotenv import load_dotenv
load_dotenv()

from services.twilio import client
from services.cache import get_agent_metadata, get_all_agents
from services.chat.chat import similarity_search, get_embedding
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from app.core.config import settings

from composio import Composio

# Initialize the Composio client
client = Composio()

In [ ]:
""" Get all connected apps and accounts """
""" store connection ID to each user_id in supabase """


# Get the list of integrations (connected apps)
integrations = client.integrations.get()
print("Connected Apps:")
for integration in integrations:
    print(f"• App: {integration.appName}")
    print(f"  ID: {integration.id}")

# Get the list of connected accounts (existing connections)
connected_accounts = client.connected_accounts.get()
print("\nExisting Connections:")
for account in connected_accounts:
    print(f"• ID: {account.id}")
    print(f"  App: {account.appName}")
    print(f"  Status: {account.status}")


In [ ]:
from services.composio import get_calendar_slots

get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv")

In [75]:
from composio import Composio

# Initialize the Composio client
composio_client = Composio()

# Create a new entity with a custom ID
new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")


In [ ]:
""" checks if entity already exists """

existing_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
existing_connections = existing_entity.get_connections()

if existing_connections:
    print("Entity already exists with connections:", existing_connections)
else:
    print("Entity is newly created or has no connections")


# # Check for connected accounts for the new entity
# try:
#     connected_account = new_entity.get_connection(app="outlook")
#     print(f"Connected Outlook account: {connected_account}")
# except Exception as e:
#     print("No connected Slack account found")

# # Perform actions using the new entity
# actions = new_entity.get_tools(app_name="github")

existing_connections

### Outlook

In [ ]:
""" this function will be triggered in endpoint /composion/new_connection """
""" then to store the connection ID to the user_id in supabase """
""" frontend to have button to "generate auth link", response from this endpoint will be the redirect url """
### entity ID will be created for each user. 
""" {"app": "outlook", "connectedAccountId" : "xxx", "client_id" : "xxx" } """

"""Auth"""
from composio import ComposioToolSet, App
from composio import Composio
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

# Initialize the Composio client
composio_client = Composio()

# # Create a new entity with a custom ID
# new_entity = composio_client.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

toolset = ComposioToolSet()
entity = toolset.get_entity(id="user_2mmXezcGmjZCf88gT2v2waCBsXvww")

# request = entity.initiate_connection(App.OUTLOOK)
# ### background task save request.connectedAccountId to integrations table in supabase 
# ### then return the below redirect url to frontend
# print(f"Open this URL to authenticate: {request.redirectUrl}")


In [ ]:
tools = toolset.get_tools(actions=[Action.OUTLOOK_LIST_EVENTS])

tools

In [ ]:
tools = toolset.get_tools(apps=[App.OUTLOOK])

tools

In [62]:
openai_client = OpenAI()

today = datetime.now().strftime("%Y-%m-%d")
task = f"get calendar slots for next week, today is {today}"

# Update the tools to include search capability
tools = toolset.get_tools(actions=[
    Action.OUTLOOK_OUTLOOK_UPDATE_CALENDAR_EVENT])


# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = toolset.handle_tool_calls(search_response)
search_result


In [ ]:
search_result

### Shopify

In [ ]:
"""Auth"""
""" shopify not working :("""

from composio import ComposioToolSet, Composio, App
from dotenv import load_dotenv
load_dotenv()

client = Composio()
toolset = ComposioToolSet()
entity = toolset.get_entity()
request = entity.initiate_connection(App.SHOPIFY, connected_account_params={"shop": "fdi1gh-xg"})

print(f"Open this URL to authenticate: {request.redirectUrl}")


### Notion

In [ ]:
"""Auth"""

from composio import ComposioToolSet, App

toolset = ComposioToolSet(entity_id="Jessica")
entity = toolset.get_entity()
request = entity.initiate_connection(App.NOTION)

print(f"Open this URL to authenticate: {request.redirectUrl}")


In [8]:
from composio_openai import ComposioToolSet, Action
from openai import OpenAI

openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="Jessica")

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")
task = "get the DB id for DevOps DB"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_SEARCH_NOTION_PAGE])

# First, let's search for your database
search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)

db_id = search_result[0]['data']['response_data']['results'][0]['id']
print("DB ID:", db_id)

task = f"get the page to do's in DB with id {db_id}"

# Update the tools to include search capability
tools = composio_toolset.get_tools(actions=[
    Action.NOTION_QUERY_DATABASE])

search_response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Search for a database called DevOps DB in my Notion workspace"},
    ],
)
search_result = composio_toolset.handle_tool_calls(search_response)



In [ ]:
print(search_result)


### Google Calendar

In [ ]:
from composio_openai import ComposioToolSet, Action
from composio import Composio
from openai import OpenAI
from datetime import datetime 

from dotenv import load_dotenv
load_dotenv()

client = Composio()
openai_client = OpenAI()
composio_toolset = ComposioToolSet(entity_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

GOOGLECALENDAR_CREATE_EVENT: 'Action'
GOOGLECALENDAR_DELETE_EVENT: 'Action'
GOOGLECALENDAR_DUPLICATE_CALENDAR: 'Action'
GOOGLECALENDAR_FIND_EVENT: 'Action'
GOOGLECALENDAR_FIND_FREE_SLOTS: 'Action'
GOOGLECALENDAR_GET_CALENDAR: 'Action'
GOOGLECALENDAR_GET_CURRENT_DATE_TIME: 'Action'
GOOGLECALENDAR_LIST_CALENDARS: 'Action'
GOOGLECALENDAR_PATCH_CALENDAR: 'Action'
GOOGLECALENDAR_QUICK_ADD: 'Action'
GOOGLECALENDAR_REMOVE_ATTENDEE: 'Action'
GOOGLECALENDAR_UPDATE_EVENT: 'Action'

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_LIST_CALENDARS])
# task = "find free slots for next week"

# tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_GET_CALENDAR])
# task = "fetch calendar for id michael@flowon.ai"

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_FIND_FREE_SLOTS])
today = datetime.now().strftime("%Y-%m-%d")

task = f"Search for free slots for the next 14 days for id michael@flowon.ai. Today's date is {today}."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)

result = composio_toolset.handle_tool_calls(response)

In [ ]:
from services.composio import get_calendar_slots, find_free_slots

await get_calendar_slots("user_2mmXezcGmjZCf88gT2v2waCBsXv","outlook")



In [2]:

tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

task = "Checkup appointment on Friday, November 29th, 2024 at 2:30 PM'Create"

today = datetime.now().strftime("%Y-%m-%d")

response = openai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    tools=tools,
    messages=[
        {"role": "system", "content": f"You are a helpful assistant. Today's date is {today}."},
        {"role": "user", "content": task},
    ],
)
result = composio_toolset.handle_tool_calls(response)
print(result)


NameError: name 'composio_toolset' is not defined

In [4]:
from services.composio import book_appointment_composio
import logging

logger = logging.getLogger(__name__)


In [13]:
async def book_appointment_composio(appointment_details: str, user_id: str) -> str:
    logger.info(f"Starting appointment booking for user_id: {user_id}")
    print(f"func book_appointment_composio: booking")

    entity_id = user_id
    openai_client = OpenAI()
    today = datetime.now().strftime("%Y-%m-%d")
    current_time = datetime.now().strftime("%H:%M")

    logger.debug(f"Initializing ComposioToolSet for entity_id: {entity_id}")
    composio_toolset = ComposioToolSet(entity_id=entity_id)

    logger.debug("Getting calendar tools")
    tools = composio_toolset.get_tools(actions=[Action.GOOGLECALENDAR_CREATE_EVENT])

    task = f"Book an appointment on the user's calendar with the following details: {appointment_details}. Today's date is {today}. Current time is {current_time}."
    logger.debug(f"Created task: {task}")

    try:
        logger.debug("Making OpenAI API call")
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            tools=tools,
            messages=[
                {"role": "system", "content": f"You are a helpful assistant helping to manage the user's calendar. Today's date is {today}. Current time is {current_time}."},
                {"role": "user", "content": task},
            ],
        )
        logger.debug("OpenAI API call successful")
        
        logger.debug("Handling tool calls")
        result = composio_toolset.handle_tool_calls(response)
        
        if result[0]['successfull'] == 'True':
            logger.info("Event created successfully")
            print("Event created successfully")
            return "Appointment booked successfully."
        else:
            logger.error(f"Event creation failed: {result[0]['error']}")
            print("Event creation failed")
            print(result[0]['error'])
            return "Appointment booking failed."
    except Exception as e:
        logger.error(f"Unexpected error during appointment booking: {str(e)}")
        return "Appointment booking failed due to an unexpected error."

results = await book_appointment_composio("Checkup appointment on Friday, November 29th, 2024 at 2:30 PM", "user_2mmXezcGmjZCf88gT2v2waCBsXv")
results

[2024-11-27 22:17:04,192][INFO] Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


func book_appointment_composio: booking


[2024-11-27 22:17:07,242][INFO] Executing `GOOGLECALENDAR_CREATE_EVENT` with params={'summary': 'Checkup appointment', 'start_datetime': '2024-11-29T14:30:00', 'event_duration': '1h', 'calendar_id': 'primary'} and metadata={} connected_account_id=None
[2024-11-27 22:17:08,924][INFO] Got response={'successfull': True, 'data': {'response_data': {'kind': 'calendar#event', 'etag': '"3465491657466000"', 'id': 'l5i36jcuk7ulavd19grp9ktap4', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=bDVpMzZqY3VrN3VsYXZkMTln...


[{'successfull': True,
  'data': {'response_data': {'kind': 'calendar#event',
    'etag': '"3465491657466000"',
    'id': 'l5i36jcuk7ulavd19grp9ktap4',
    'status': 'confirmed',
    'htmlLink': 'https://www.google.com/calendar/event?eid=bDVpMzZqY3VrN3VsYXZkMTlncnA5a3RhcDQgbWljaGFlbEBmbG93b24uYWk',
    'created': '2024-11-27T22:17:08.000Z',
    'updated': '2024-11-27T22:17:08.733Z',
    'summary': 'Checkup appointment',
    'creator': {'email': 'michael@flowon.ai', 'self': True},
    'organizer': {'email': 'michael@flowon.ai', 'self': True},
    'start': {'dateTime': '2024-11-29T14:30:00Z', 'timeZone': 'UTC'},
    'end': {'dateTime': '2024-11-29T15:30:00Z', 'timeZone': 'UTC'},
    'iCalUID': 'l5i36jcuk7ulavd19grp9ktap4@google.com',
    'sequence': 0,
    'reminders': {'useDefault': True},
    'eventType': 'default'}},
  'error': None}]

In [17]:
results[0]['successfull']

True